<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

**Описание проекта**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Инструкция по выполнению проекта**

- Загрузите и изучите данные.
- Ответьте на вопрос и обоснуйте решение. 
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

    a. Изменится. Приведите примеры матриц.

    b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
    
- Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

**Описание данных**

- Набор данных находится в файле /datasets/insurance.csv. Скачать датасет.
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
! pip install -q phik

In [2]:
# Импортируем необходимые модули
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from ydata_profiling import ProfileReport
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'ydata_profiling'

In [ ]:
# Снимем ограничение на количество столбцов
pd.set_option('display.max_columns', None)

# Снимем ограничение на ширину столбцов
pd.set_option('display.max_colwidth', None)

# Игнорируем предупреждения
pd.set_option('chained_assignment', None)

In [ ]:
# Загрузим данные
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')

In [ ]:
print(data.head(3))
print()
print('Информация датасета:')
print(data.info())
print()
print('Описание датасета:')
print(data.describe())
print()
print(f'Количествово пропусков:{data.isna().sum()}') 
print()
print(f'Количествово дубликатов:{data.duplicated().sum()}')
print()
print('Уникальные значения датасета:')
print(data.nunique())

In [ ]:
corr_matrix = data.corr()
print(corr_matrix)
fig, ax = plt.subplots(figsize=(50, 50))
sns.heatmap(corr_matrix, annot=True, cmap="Spectral", annot_kws={"fontsize":38})
fig.suptitle('Тепловая карта', fontsize=40)
plt.xticks(fontsize=30, rotation=45)
plt.yticks(fontsize=30, rotation=300)
plt.show();

Существуют разные коэффициенты корреляции, и не все из них универсальны. Например, корреляция Пирсона не подходит для категориальных признаков. Есть и более гибкие инструменты: так, коэффициент корреляции Фи находит взаимосвязь между категориальными, порядковыми и интервальными признаками. Чем ближе его значение по модулю к 1, тем сильнее связь. Используем этот передовой метод определения корреляции.

In [ ]:
data.phik_matrix() 

Хотя используя phik_matrix лучше вручную указать столбцы с количественными признаками через interval_cols, а то он иногда путается

In [ ]:
data.phik_matrix(interval_cols=['Страховые выплаты', 'Возраст', 'Зарплата', 'Члены семьи'])

In [ ]:
graph = sns.pairplot(data, hue='Страховые выплаты', diag_kind="hist")
graph.fig.suptitle("Матрица рассеяния параметров датасета", y=1);

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=data['Зарплата'], palette='BrBG_r', orient='h')
plt.title('Зарплата')
plt.show();

In [ ]:
columns = ['Возраст', 'Страховые выплаты', 'Члены семьи']
for col in columns:
    sns.boxplot(data=data[col], palette='BrBG_r', orient='h')
    plt.title(col)
    plt.show();

In [ ]:
# Для сравнения распределений выборок с разным размером используем kde
columns = ['Возраст', 'Зарплата', 'Страховые выплаты', 'Члены семьи']
for col in columns:
    plt.rcParams['figure.figsize'] = (15, 5)
    ax = sns.distplot(a=data[col], label=data[col], hist=True, kde=True, rug=False, 
                      bins=100, hist_kws={"color":"red", "alpha":1})
    ax.set_ylabel('Плотность')
    sns.set(rc={'figure.figsize':(20,5)})
    plt.show();

Видим что в столбцах Возраст и Зарплата тип данных float, хотя значения целые, так что поменяем их для уменьшения памяти и быстродеятсвия программы.

In [ ]:
# Также изучить данные можно подобным способом
ProfileReport(data, title="Данные датасета")

In [ ]:
data['Возраст'] = pd.to_numeric(data['Возраст'], downcast='integer')
data['Зарплата'] = pd.to_numeric(data['Зарплата'], downcast='integer')
data[['Зарплата', 'Возраст']] = data[['Зарплата', 'Возраст']].astype('int64')

**Итоги:** 

Загрузили и предобработали данные:
- Пропусков нет.
- Есть некоторые выбросы.
- От дубликатов избавляться не будем, вдруг это совпадающие признаки разных клиентов. Да и в принципе, учитывая конечную цель проекта, манипуляции с данными не целесообразны. Данные оставляем в исходном виде.
- Заменили типы данных в столбцах Возраст и Зарплата на более подходящий int.
- В таблице соблюден почти поровну мужчинам и женщин. 
- Средний возраст застрахованного = 31 год, а медиана = 30. 
- Судя по данным компания не занимается страхованием лиц в возрасте менее 18 и старше 65 лет. 
- Медиана зарплаты составляет чуть более 40 000 рублей.
- Видим, что Возраст и Страховые выплаты зависимы между собой больше всего.

В данных есть выбросы. Значимые нарушения не выявлены.


Итоги анализа загруженных данных

Датафрейм содержит 5000 объектов и 5 числовых признаков.
В данных нет пропусков.
Целевой признак содержит 6 категорий от 0 до 5. Вероятнее всего, категории обозначают количество страховых выплат. По мере увеличения значения категории уменьшается количество объектов в ней. Так, в категории 0 содержится 4436 объектов, в категории 1 только 423 объекта, а в категории 5 всего 1 объект.
По мере увеличения количества страховых выплат увеличивается возраст застрахован лица. Эти два показателя имеют сильную положительную корреляцию. При этом, количество страховых выплат почти не зависит от пола и других имеющихся признаков застрахованного лица.
Признаки имеют значительное различие в разбросе числовых значений. Для эффективного использования в обучении их следует стандартизировать.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:** 

Так как в задании сказано, что матрица признаков умножается на обратимую матрицу, то матрица P - квадратная. Умноженная матрица будет иметь вид X * P. Тогда фомула предсказаний:

$$
a' = XP'w
$$

$$
a' = X'w' = XPw'
$$

$$
w' = (X'^T X')^{-1} X'^T y = ((XP)^TXP)^{-1})(XP)^Ty
$$

Используем её в вычислениях:

$$
a' = XP((XP)^TXP)^{-1})(XP)^Ty
$$

Воспользуемся свойствами матриц:

$$
(AB)^{-1} = B^{-1}A^{-1}
$$
$$
(AB)^T = B^TA^T
$$
$$
AA^{-1} = E
$$
$$
AE = EA = A
$$
$$
𝐴(𝐵𝐶)=(𝐴𝐵)𝐶
$$

Получаем уравнение:

$$
a' = XP(P^TX^TXP)^{-1})P^TX^Ty = XPP^{-1}(P^TX^TX)^{-1}P^TX^Ty = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$PP^{-1}$ и $(P^T)^{-1}P^T$ дают единичные матрицы, которые исключаем из уровнения. Получаем следующее:

$$
a' = X^TX^{-1}X^Ty = Xw
$$

Следовательно:
$$
a = a' = Xw ==> 𝑋𝑃𝑤𝑝=𝑋𝑤
$$

Теперь, чтобы было верно это:
    
    
$$
XP w_p = X w
$$    
    
    
Нужно чтобы было верно это:   
    
    
    
$$
P w_p = w
$$    
       
    
Перемножим обе части равенства на $P^{-1}$, и получим что   $
w_p = P^{-1}w
$    


Вот и нашли соотношение наших весов.   

**Итоги:** 

Значение предсказания не меняется если просходит перемножение признаков на обратимую матрицу.

## Алгоритм преобразования

**Алгоритм**

1. В качестве алгоритма преобразования мы примем домножение обучающих признаков X на случайную обратимую матрицу P размером 4х4. Будем генерировать рандомную матрицу пока не получим обратимую. Ведь если матрица будет необратима, мы не сможем расшифровать наши данные. 
2. Затем мы создадим модель линейной регрессии до преобразования и посчитаем метрику R2. 
3. После этого мы умножим исходные признаки на обратимую матрицу P и на основе полученных значений вновь посчитаем метрику R2.
4. Ожидается, что метрики R2 до и после преобразования будут равны. 
5. Для возврата к исходной матрице, умножим новую матрицу на обратную матрицу.

**Обоснование**

На основании выводов, полученных в пункте 2 данного проекта предполагается, что умножение треннировочной выборки с нецелевыми аттрибутами и аналогичной тестовой выборки на случайную обратимую матрицу даст такой же результат, что и без умножения на подобную матрицу.

**Итоги:**

Написали и обосновали алгоритм преобразования исходных данных, чтобы по ним было сложно восстановить персональную информацию, без ухудшения качества моделей машинного обучения.

## Проверка алгоритма

In [ ]:
# Разделим наши данные
features = data.drop(["Страховые выплаты"], axis=1)
target = data["Страховые выплаты"]
features.shape, target.shape

In [ ]:
# Создаем выборки обуч и тест
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=777)

In [ ]:
def func(dim):
    global transform_matrix
    global inv_matrix
    transform_matrix = np.random.rand(dim, dim)
    try: 
        inv_matrix = np.linalg.inv(transform_matrix)
        return transform_matrix, inv_matrix
    except:
        func(dim) 

In [ ]:
# Проверим её на обратимость
try:
    func(4)
    print('\nОбратная матрица к заданной существует')
except:
    print('\nОбратная матрица к заданной не существует')

In [ ]:
inv_matrix

In [ ]:
features.head(3)

In [ ]:
model = LinearRegression(normalize=True).fit(features_train, target_train)
predictions = model.predict(features_test)
r2_score_value = r2_score(target_test, predictions)
print(f"R2_score: {r2_score_value:.2f}")

In [ ]:
# Создадим датасет, в котором умножим данные на обратимую матрицу
features_matrix = features_train @ inv_matrix
features_matrix.head(3)

In [ ]:
model = LinearRegression(normalize = True).fit(features_matrix, target_train)
predictions = model.predict(features_matrix)
r2_score_value = r2_score(target_train, predictions)
print(f"R2_score: {r2_score_value:.2f}")

**Итоги:**

Поскольку значения метрики R2 сошлись, мы можем считать, что успешно защитили данные пользователей, применив умножение на матрицу, при этом не потеряв качество модели.

## Итоговый вывод

В результате выполнения проекта решена задача и достигнута цель - разработан такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснована корректность его работы. При преобразовании данных качество модели машинного обучения не ухудшилось.

**Выполнены следующие этапы:**

1. Загрузили и предобработали данные:
- Пропусков нет.
- Есть некоторые выбросы.
- От дубликатов избавляться не будем, вдруг это совпадающие признаки разных клиентов. Да и в принципе, учитывая конечную цель проекта, манипуляции с данными не целесообразны. Данные оставляем в исходном виде.
- Заменили типы данных в столбцах Возраст и Зарплата на более подходящий int.
- В таблице соблюден почти поровну мужчинам и женщин. 
- Средний возраст застрахованного = 31 год, а медиана = 30. 
- Судя по данным компания не занимается страхованием лиц в возрасте менее 18 и старше 65 лет. 
- Медиана зарплаты составляет чуть более 40 000 рублей.
- Видим, что Возраст и Страховые выплаты зависимы между собой больше всего.

2. Выяснили, что значение предсказания не меняется если просходит перемножение признаков на обратимую матрицу.


3. Написали и обосновали алгоритм преобразования исходных данных, чтобы по ним было сложно восстановить персональную информацию, без ухудшения качества моделей машинного обучения.

4. Проверили алгоритм:
- Данные разбили.
- Умножили данные на обратную матрицу.
- Обучили модель линейной регрессии.
- Поскольку значения метрики R2 сошлись, мы можем считать, что успешно защитили данные пользователей, применив умножение на матрицу, при этом не потеряв качество модели.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования